In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
bs=52 # 52 - Jeremey, 20 - default

In [4]:
bptt= 70 #70 - Jeremey, 35 - default

In [5]:
def create_data_lm(tokens,bs):
    import itertools
    # Collapse into a single large array
    tokens=np.asarray(list (itertools.chain(*tokens)))
    # How many batches
    n_batch=len(tokens)//bs
    # Truncate to exclude the ones at the end
    tokens=tokens[:bs*n_batch]
    # Reshape
    tokens=tokens.reshape(bs,-1)
    return tokens

In [6]:
train_tokens=create_data_lm(df_train['tokens'],bs)
valid_tokens=create_data_lm(df_valid['tokens'],bs)
test_tokens=create_data_lm(df_test['tokens'],bs)
train_tokens.shape, valid_tokens.shape, test_tokens.shape

((52, 396650), (52, 43895), (52, 143259))

In [7]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [8]:
n_batch=train_tokens.shape[1]
for i in range(0,n_batch,bptt):
    seq_len=min(bptt,n_batch-1-i)
    x=train_tokens[:,i:i+seq_len]
    y=train_tokens[:,i+1:i+1+seq_len]
    print (x.shape, y.shape)
    break

(52, 70) (52, 70)


In [9]:
x.shape,y.shape

((52, 70), (52, 70))

In [10]:
x,y

(array([[   41,    42,    43,    40, ...,     2,  1896,   463,     4],
        [ 1071,    23,    67,    39, ...,     2,    98,    11,    63],
        [   54,    34,     7,   190, ...,     2,    87,    34,  5998],
        [    7,  1213,     8,   453, ...,   406,     4,     2,    46],
        ...,
        [   60,   126,   931,     6, ...,   114,    39,   599,    63],
        [    2, 15094,   132,    65, ...,    56,  1350,   209,   281],
        [11783,   658,   318,     3, ...,     6,    64,    79,  1268],
        [   42,    43,    40,     2, ...,   708,   111,     4,     2]]),
 array([[   42,    43,    40,    13, ...,  1896,   463,     4,     2],
        [   23,    67,    39,    18, ...,    98,    11,    63,    28],
        [   34,     7,   190,  1732, ...,    87,    34,  5998,    47],
        [ 1213,     8,   453,    21, ...,     4,     2,    46,     7],
        ...,
        [  126,   931,     6,   126, ...,    39,   599,    63,   314],
        [15094,   132,    65,   411, ...,  1350, 

In [11]:
x.shape,y.shape

((52, 70), (52, 70))

In [12]:
train_tokens.shape

(52, 396650)

## pre-trained GLOVE

In [13]:
PATH_GLOVE='/home/kirana/Documents/phd/data/pre-trained/glove/glove.6B'

In [14]:
ls {PATH_GLOVE}

glove_50_glove_100.tgz  glove.6B.300d.txt  wordvectors_glove50
glove.6B.100d.txt       glove.6B.50d.txt
glove.6B.200d.txt       word2idx_glove50


In [15]:
def load_glove_vectors(path=f'{PATH_GLOVE}/glove.6B.50d.txt'):
    f=open(f'{path}','r')
    glovelines=f.readlines()
    word2idx={}
    idx=0
    wordvectors={}
    for line in glovelines:
        line=line.split(' ')
        vector=np.asarray(line[1:]).astype(np.float32)
        word2idx[line[0]]=idx
        idx=idx+1
        wordvectors[line[0]]=vector
    return word2idx,wordvectors

In [18]:
word2idx_glove,wordvectors_glove=load_glove_vectors()
import pickle
pickle.dump(word2idx_glove,open(f'{PATH_GLOVE}/word2idx_glove50','wb'))
pickle.dump(wordvectors_glove,open(f'{PATH_GLOVE}/wordvectors_glove50','wb'))

In [19]:
word2idx_glove=pickle.load(open(f'{PATH_GLOVE}/word2idx_glove50','rb'))
wordvectors_glove=pickle.load(open(f'{PATH_GLOVE}/wordvectors_glove50','rb'))

In [20]:
len(word2idx_glove)

400000

In [21]:
[*word2idx_glove.keys()][:5]

['the', ',', '.', 'of', 'to']

In [22]:
wordvectors_glove['the'].shape

(50,)

In [23]:
glove_mtx=np.array([wordvectors_glove[key] for key in [*word2idx_glove.keys()]])

In [24]:
glove_mtx.shape

(400000, 50)

In [25]:
# In our list but not in Glove
len(set(itos).difference(set([*word2idx_glove.keys()])))

5072

In [26]:
len(set(itos).difference(set([*word2idx_glove.keys()])))/len(itos)

0.0845728006402988

In [27]:
glove_mtx=np.zeros((len(itos),glove_mtx.shape[1]))

In [37]:
wordvectors_glove['the']

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01, -4.4457e-02, -4.9688e-01, -1.7862e-01,
       -6.6023e-04, -6.5660e-01,  2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,  1.1658e-02,
        1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01, -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01,
       -1.9181e-01, -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,  4.0071e+00, -1.8594e-01,
       -5.2287e-01, -3.1681e-01,  5.9213e-04,  7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,  1.8785e-01,  2.7849e-03, -1.8411e-01,
       -1.1514e-01, -7.8581e-01], dtype=float32)

In [33]:
enc_wgts = glove_mtx # converts np.ndarray from torch.FloatTensor.output shape: (238462, 400)
row_m = enc_wgts.mean(0) # returns the average of the array elements along axis 0. output shape: (400,)

In [34]:
for i,word in enumerate(itos):
    try:
        glove_mtx[i]=wordvectors_glove[word]
    except:
        glove_mtx[i]=row_m

In [36]:
glove_mtx[3]

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01, -4.4457e-02, -4.9688e-01, -1.7862e-01,
       -6.6023e-04, -6.5660e-01,  2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,  1.1658e-02,
        1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01, -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01,
       -1.9181e-01, -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,  4.0071e+00, -1.8594e-01,
       -5.2287e-01, -3.1681e-01,  5.9213e-04,  7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,  1.8785e-01,  2.7849e-03, -1.8411e-01,
       -1.1514e-01, -7.8581e-01])

In [38]:
enc_wgts.shape, len(itos)

((59972, 50), 59972)

In [39]:
n_emb=enc_wgts.shape[1]

In [40]:
new_w=enc_wgts

In [41]:
new_w.shape

(59972, 50)

## Model Architecture

In [46]:
from torch import nn
import torch
from adaptive import *
from torch.autograd import Variable

In [42]:
n_inp=len(itos)
n_emb=50 #650
n_hidden=50 #650
n_layers=3
dropout=0.5
wd=1e-7

In [43]:
class language_model (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,adaptive_log_softmax=True,tie_weights=True):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx
        self.adaptive_log_softmax,self.tie_weights=adaptive_log_softmax,tie_weights
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
            
        if self.adaptive_log_softmax is False:
            self.criterion=nn.CrossEntropyLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer
        self.encoder=nn.Embedding(self.n_inp,self.n_emb)
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=False)
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        if self.adaptive_log_softmax:
            # Adaptive Log Softmax Loss
            self.adaptive_softmax=AdaptiveLogSoftmaxWithLoss(self.n_hidden,
                                    self.n_inp,
                                    cutoffs=[round(self.n_inp/15),3*round(self.n_inp/15)],
                                    div_value=4,
                                    get_full_prob=True)
        else:
            self.decoder=nn.Linear(self.n_hidden,self.n_inp)
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
        if self.tie_weights:
            self.decoder.weight.requires_grad=False
        
    
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True
        if self.tie_weights:
            self.decoder.weight.requires_grad=True
        
    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
        if self.tie_weights:
            self.decoder.weight=self.encoder.weight
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        out,new_hidden=self.lstm(embs,self.hidden)
        out=self.dropout_op(out)
         # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        if self.adaptive_log_softmax:
            out=out.reshape(out.size(0)*out.size(1),out.size(2))        # output is of shape n_batch * n_seq * n_hidden
      
            out=self.adaptive_softmax(out,Yb.view(-1))
            loss=out.loss
            preds=out.output_full
        else:
            #import pdb
            #pdb.set_trace()
            preds=self.decoder(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))
            loss=self.criterion(preds,Yb.contiguous().view(-1))

        return preds, loss  

In [44]:
def accuracy_multinomial(preds,actual):
    preds=preds.max(1)[1]
    correct=preds==actual
    return correct.float().sum()/len(correct)

In [47]:
device="cuda:0"
model=language_model(n_inp,n_emb,n_hidden,n_layers,False,bs,device,0.05,0.5,0.5,new_w,False,True)


initializing


In [48]:
new_w.shape

(59972, 50)

In [49]:
model.encoder.weight.data.shape,model.decoder.weight.data.shape

(torch.Size([59972, 50]), torch.Size([59972, 50]))

In [50]:
nn.Linear(n_hidden,n_inp).weight.data.shape, n_hidden, n_inp

(torch.Size([59972, 50]), 50, 59972)

### Test if model forward works

In [51]:
model.decoder.weight.shape,model.encoder.weight.shape

(torch.Size([59972, 50]), torch.Size([59972, 50]))

In [52]:
x.shape

(52, 70)

In [53]:
if 1==0:
    model.forward(torch.LongTensor(x),torch.LongTensor(y))

In [54]:

model=model.to(device)

## Learner

In [55]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb)
        
       
            
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        n_batch=iterator.shape[1]
        epoch_loss,epoch_acc,i=0,0,0
        self.model.gen_hidden()
        for k,i in enumerate(range(0,n_batch,self.bptt)):
            seq_len=min(bptt,n_batch-1-i)
            Xb=train_tokens[:,i:i+seq_len]
            Yb=train_tokens[:,i+1:i+1+seq_len]
            Xb=torch.LongTensor(Xb)
            Yb=torch.LongTensor(Yb)
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
        
        if 1==0:
            lr /= 4.0
            # Freeze all the layers initially
            for param in resnet.parameters():
                param.requires_grad=False
            torch.save(resnet,'resnet')
            torch.save(resnet.state_dict(),'resnet_state_dict')
            resnet.load_state_dict(torch.load('resnet_state_dict'))
            for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr  
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [56]:
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_multinomial

In [57]:
n_batch=np.int(np.ceil(train_tokens.shape[1]/bptt))
n_batch

5667

In [58]:
model.freeze_embedding()

In [59]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(False, False)

In [60]:
learner=Learner(model,optimizer,accuracy_multinomial,device,bptt,500,0.25)

In [61]:
model.n_inp

59972

In [62]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:500 8.975491941452026  0.0061406596222077494
Batch:1000 8.692644960403442  0.009156868540361756
Batch:1500 8.475224107424419  0.0120673997871927
Batch:2000 8.28836444926262  0.015320192958999541
Batch:2500 8.122718526649475  0.018832748111209368
Batch:3000 7.973838469028473  0.022595605326709726
Batch:3500 7.838934770175389  0.026472607004231707
Batch:4000 7.715929921030998  0.030334548054648622
Batch:4500 7.604136815706889  0.034098231149559274
Batch:5000 7.502896773529053  0.03796818856553291
Batch:5500 7.409750179204074  0.042435815985510335
Epoch:0 Loss:7.3808314409771025
Batch:500 6.142567819595337  0.12176758889853954
Epoch:0 Loss:7.3808314409771025 Accuracy:0.04397693012322231 Loss:6.140104875610206 Accuracy:0.12157229394718791
Batch:500 6.3805231208801265  0.1016934098303318
Batch:1000 6.341601963043213  0.10463352094590664
Batch:1500 6.312938816070557  0.10719084799289703
Batch:2000 6.287314788818359  0.10933077526092529
Batch:2500 6.266097012138367  0.11053275354504585


In [63]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 6.078976338386536  0.12054286405444145
Batch:1000 6.07230401468277  0.12041044642031193
Batch:1500 6.07150251865387  0.12037656359374523
Batch:2000 6.06898297905922  0.12059533644840122
Batch:2500 6.0674547111511234  0.12052352325022221
Batch:3000 6.066049376328786  0.12055907270809015
Batch:3500 6.064398186411176  0.12063784036466053
Batch:4000 6.063378535032272  0.12069279520772397
Batch:4500 6.0620852540334065  0.12081197253200743
Batch:5000 6.061506959438324  0.12086214955151081
Batch:5500 6.060173147201538  0.12096499166569927
Epoch:0 Loss:6.059898841267461
Batch:500 5.84247246837616  0.14542088744044304
Epoch:0 Loss:6.059898841267461 Accuracy:0.12098373994993902 Loss:5.840349059736139 Accuracy:0.14526680280622303


In [64]:
torch.save(model.state_dict(),f'{PATH}/inter/glove50_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove50_learner_state_dict')

In [65]:
model.unfreeze_embedding()

In [66]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(True, True)

In [67]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:500 5.923489140510559  0.1383637427687645
Batch:1000 5.839897509098053  0.14385824939608574
Batch:1500 5.787385323842367  0.14753407403826713
Batch:2000 5.745237706899643  0.15060275558382272
Batch:2500 5.712291193008423  0.15279110733270646
Batch:3000 5.685442800998688  0.15454387297232947
Batch:3500 5.662019091742379  0.1561941215608801
Batch:4000 5.642152305483818  0.15749368988722562
Batch:4500 5.624449687639872  0.15870306105746163
Batch:5000 5.609346181678772  0.1597312722891569
Batch:5500 5.5948268599943685  0.16073991865190593
Epoch:0 Loss:5.590528269096857
Batch:500 5.153270504951477  0.1958033037483692
Epoch:0 Loss:5.590528269096857 Accuracy:0.16102258566250197 Loss:5.152303151156533 Accuracy:0.19555949147618368
Batch:500 5.448493457794189  0.17179890966415406
Batch:1000 5.437963190078736  0.17195907419919967
Batch:1500 5.434427093187968  0.17187345155080158
Batch:2000 5.427937515258789  0.17227857970446347
Batch:2500 5.422540602493286  0.1724734148979187
Batch:3000 5.4

In [68]:
torch.save(model.state_dict(),f'{PATH}/inter/glove50_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove50_learner_state_dict')

In [69]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:500 5.3223634014129635  0.18038077726960183
Batch:1000 5.316381345748901  0.18031621696054936
Batch:1500 5.316147819201151  0.18011868943770726
Batch:2000 5.313197108983993  0.18028063988685608
Batch:2500 5.310846315383911  0.18024330460429192
Batch:3000 5.308963719526926  0.18025476981699468
Batch:3500 5.306856641088213  0.18038344591004507
Batch:4000 5.305174709796906  0.18044739807397128
Batch:4500 5.303480764283075  0.18055757814645768
Batch:5000 5.302480280590057  0.18063506286144257
Batch:5500 5.300642538244074  0.18078322469104421
Epoch:0 Loss:5.300245186148959
Batch:500 4.988265664100647  0.20725715133547784
Epoch:0 Loss:5.300245186148959 Accuracy:0.18078733923849266 Loss:4.987243751970015 Accuracy:0.20695774364129207
Batch:500 5.297073210716247  0.18213572204113007
Batch:1000 5.2914591135978695  0.18196401883661747
Batch:1500 5.291667382876078  0.1816893852253755
Batch:2000 5.288867577075958  0.18183173863589763
Batch:2500 5.286954933929444  0.1817383594930172
Batch:3000

In [70]:
torch.save(model.state_dict(),f'{PATH}/inter/glove50_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove50_learner_state_dict')

In [71]:
np.exp(3.9), np.exp(4.96)

(49.40244910553017, 142.5937958969891)

In [80]:
if 1==0:
    learner.lr_find(1e-4,1e-1,valid_tokens,n_batch)
    learner.plot_lrs(50)
    len(learner.lrs)
    len(learner.losses)

Batch:500 7.944759424209595  0.03532285663112998
Batch:1000 7.877773783683777  0.03720142808556557
Batch:1500 7.802010853131613  0.03968857092410326
Batch:2000 7.722838351964951  0.04160571377538145
Batch:2500 7.658665308570862  0.043098285208642485


## Transfer Learning to predict RNN